In [14]:
# !pip install openpyxl

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [2]:
df = pd.read_csv("/data/Augmented_data.csv")
df.shape

(234501, 18)

In [3]:
df[df["Exited"] == 1].shape[0] /df.shape[0]

0.14290770615050682

In [4]:
df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned,date
0,15572489,HEALEY,1720.007287,Spain,Female,73,29,393987.727351,1,1,1,1.0,0,0,3,DIAMOND,2531.0,1-6-2022
1,15800464,PARTIDA,1518.782461,Spain,Male,53,15,274502.817582,2,1,0,1.0,1,0,4,SILVER,1725.0,1-2-2022
2,15580134,Crawford,502.950000,Spain,Male,27,2,184535.891500,1,1,1,1.0,0,0,3,DIAMOND,711.0,1-1-2020
3,15074588,PINKERTON,1468.954354,Spain,Male,49,15,0.000000,1,1,0,1.0,0,0,5,SILVER,1512.0,1-1-2022
4,15982065,ABERNATHY,1683.914742,France,Male,49,30,0.000000,2,1,0,1.0,1,0,3,DIAMOND,1296.0,1-6-2022


In [5]:
cities = pd.read_excel("US_cities.xlsx")
# companies = pd.read_csv("us_companies.csv")

In [6]:
state_city = {}
for st, ct in zip(cities["state"], cities["city"]):
    if st not in state_city:
        state_city[st] = [ct]
    else:
        state[cit]

,rank,city,state,population,2000-2013 growth
0,1,New York,New York,8405837,0.048
1,2,Los Angeles,California,3884307,0.048
2,3,Chicago,Illinois,2718782,-0.061
3,4,Houston,Texas,2195914,0.110
4,5,Philadelphia,Pennsylvania,1553165,0.026
...,...,...,...,...,...
995,996,Weslaco,Texas,37093,0.288
996,997,Keizer,Oregon,37064,0.144
997,998,Spanish Fork,Utah,36956,0.781
998,999,Beloit,Wisconsin,36888,0.029
